In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [2]:
dfTest = pd.read_csv('../data/test.csv',usecols=['prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price'])
dfTrain = pd.read_csv('../data/train.csv',usecols=['prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price'])

In [ ]:
together = pd.concat([dfTest,dfTrain],axis=0)

In [4]:
columnsToAvgPerPropID = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price']
groupedMean = together.groupby('prop_id')[columnsToAvgPerPropID].mean()
groupedMean = groupedMean.add_prefix("mean_")
groupedMean.to_csv("meansTotal.csv")

In [2]:
def joinMean(df):
    df = df.merge(right=meanDF,how="left",on="prop_id",validate="many_to_one")
    return df

In [3]:
meanDF= pd.read_csv("meansTotal.csv")

In [4]:
print(len(meanDF))

136886


In [5]:
mms = MinMaxScaler()
data = meanDF.drop(['prop_id','mean_prop_review_score','mean_prop_location_score2'],axis=1)
print(data.columns)
mms.fit(data)
data_transformed = mms.transform(data)
km = KMeans(n_clusters=6)
km = km.fit(data_transformed)

Index(['mean_prop_starrating', 'mean_prop_brand_bool',
       'mean_prop_location_score1', 'mean_prop_log_historical_price'],
      dtype='object')


E:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
dummiesCluster = pd.get_dummies(km.labels_,prefix = "cluster_")

In [7]:
meanDF = pd.concat([meanDF,dummiesCluster],axis=1)

In [8]:
df = pd.read_csv('../data/test.csv')

In [9]:
df = joinMean(df)

In [10]:
df.to_csv('../data/testWithMean.csv',index=False)

In [10]:
df = pd.read_csv('../data/train.csv')
df = joinMean(df)
df.to_csv('../data/trainWithMean.csv',index=False)

Split train in train and test

In [11]:
df = shuffle(df,random_state = 10)

In [12]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
df.drop(df.tail(numOfRowsTest).index,inplace=True)

In [13]:
X_test.to_csv('../data/20PercentTestWithMean.csv',index=False)
df.to_csv('../data/80PercentTrainWithMean.csv',index=False)